In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime


import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import os
# Set logging to suppress unwanted messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress INFO, WARNING, and ERROR logs
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations

# Limit TensorFlow to use the first GPU (optional, based on your setup)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the first GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Prevent memory pre-allocation
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

import logging
logging.getLogger("cmdstanpy").setLevel(logging.CRITICAL)

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_predict

from datetime import datetime

import os
import logging
import tensorflow as tf

# Suppress TensorFlow INFO, WARNING, and ERROR logs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Set TensorFlow log level
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN optimizations

# Override logging filter to block all TensorFlow logs
class BlockAllFilter(logging.Filter):
    def filter(self, record):
        return False  # Prevent any logs from being processed

# Apply the filter to TensorFlow and absl logging
logging.getLogger("tensorflow").addFilter(BlockAllFilter())
logging.getLogger("absl").addFilter(BlockAllFilter())

# Limit TensorFlow to use the first GPU (if available)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)  # Prevent memory pre-allocation
        print("Using GPU:", gpus[0])
    except RuntimeError as e:
        print("Error initializing GPU:", e)
else:
    print("No GPU detected. TensorFlow will use the CPU.")

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Num GPUs Available: 1


In [4]:
from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [14]:
f.predictions('nvda')

NVDA 5m Interval Timestamp: 2024-12-17 05:59:35 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,up
kelly_1:2.5,0.277265,0.322662,0.26561,0.137029
prob_up,0.316897,0.337673,0.314317,0.625
prob_static,0.3525,0.374936,0.385657,0.375
prob_down,0.330604,0.287391,0.300025,0.0
precision,0.483761,0.516187,0.475436,0.383592
recall,0.583505,0.591753,0.618557,0.355236
f1,0.528972,0.551393,0.537634,0.36887
support,"[487.0, 485.0, 487.0]","[487.0, 485.0, 487.0]","[487.0, 485.0, 487.0]","[487.0, 485.0, 487.0]"


NVDA 15m Interval Timestamp: 2024-12-17 05:59:39 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,static,up
kelly_1:2.5,0.11625,0.284091,0.308434,0.129487
prob_up,0.429096,0.355114,0.322998,0.5
prob_static,0.401487,0.401853,0.400014,0.375
prob_down,0.169416,0.243033,0.276988,0.125
precision,0.36875,0.488636,0.506024,0.378205
recall,0.36646,0.534161,0.521739,0.36646
f1,0.367601,0.510386,0.513761,0.37224
support,"[161.0, 161.0, 161.0]","[161.0, 161.0, 161.0]","[161.0, 161.0, 161.0]","[161.0, 161.0, 161.0]"


NVDA 1h Interval Timestamp: 2024-12-17 05:59:41 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,up
kelly_1:2.5,0.07706,0.066667,0.085845,0.083575
prob_up,0.326458,0.34831,0.3618,0.75
prob_static,0.223437,0.299981,0.242661,0.0
prob_down,0.450105,0.351709,0.395538,0.25
precision,0.340757,0.333333,0.347032,0.345411
recall,0.354167,0.314815,0.351852,0.330254
f1,0.347333,0.32381,0.349425,0.337662
support,"[433.0, 433.0, 432.0]","[433.0, 433.0, 432.0]","[433.0, 433.0, 432.0]","[433.0, 433.0, 432.0]"


NVDA 1wk Interval Timestamp: 2024-12-17 05:59:47 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,static,static
kelly_1:2.5,0.143103,0.066667,0.145133,0.106897
prob_up,0.306206,0.360774,0.326359,0.25
prob_static,0.40007,0.338322,0.349556,0.375
prob_down,0.293724,0.300904,0.324085,0.375
precision,0.387931,0.333333,0.389381,0.362069
recall,0.511364,0.261364,0.5,0.477273
f1,0.441176,0.292994,0.437811,0.411765
support,"[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]","[88.0, 88.0, 88.0]"


NVDA 1mo Interval Timestamp: 2024-12-17 05:59:49 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,up,up,up
kelly_1:2.5,0.131034,-0.1,-0.12,-0.05
prob_up,0.313389,0.473279,0.415628,0.5
prob_static,0.558592,0.398464,0.317918,0.375
prob_down,0.128019,0.128257,0.266454,0.125
precision,0.37931,0.214286,0.2,0.25
recall,0.55,0.157895,0.157895,0.263158
f1,0.44898,0.181818,0.176471,0.25641
support,"[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]","[19.0, 20.0, 19.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')